# Titanic w/ Pipelines
## Goal
Take what we did in V1 and expand upon it with more info/tutorials on pipelining.

## Relevant Material
[Deep dive in sklearn pipelines](https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines)  
[Simple pipeline example with scikit learn](https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976)

## Titanic Data
| Variable | Definition | Key |
| ----- | --- | --- |
| survival | Survived or not | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex | |
| age | Age in years | |
| sibsp | Num of siblings / spouses aboard | |
| parch | Num of parents / children aboard | |
| ticket | Ticket number | |
| fare | Passengar fare | |
| cabin | Cabin number | |
| embarked | Port of embarkation |   C = Cherbourg, Q = Queenstown, S = Southampton |

## Base setup

In [ ]:
%reload_ext autoreload
%autoreload 2

# custom helpers
from helpers.helper import get_splits, preprocess_titanic
# data handling
import numpy as np
import pandas as pd
# output
from termcolor import cprint
import matplotlib.pyplot as plt
import seaborn as sns

cprint('All Modules Imported!', 'green')

## Data Import

In [ ]:
train_data = pd.read_csv('./data/train.csv', index_col='PassengerId')
test_data = pd.read_csv('./data/test.csv', index_col='PassengerId')

cprint('Data Imported!', 'green')
cprint('Training Data Example:', 'cyan')
display(train_data)

## Pipelining
### Reading Notes
- For very basic feature engineering, it's good to encapsulate processing logic into functions so it can be reproduced easily.
- For purposes of optimizing and testing models, it's a good idea to create feature selectors that can be used inside a pipeline to apply transformations on single columns. Check out `TextSelector` and `NumericSelector` in helper.py
  - the selectors can used as follows `('selector', TextSelector(key='processed'))`
- Pipelines are built from pipelines. For each processing step, we can create a mini-pipeline that carries out the task/engineering we need.
  - If doing engineering on individual columns, it's important to join the engineered columns back into the dataset. `sklearn.pipeline`'s `FeatureUnion` method can be great for this.

### Basic Engineering

In [15]:
numerical_cols = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_cols = ['Pclass', 'Sex', 'Cabin', 'Embarked']
target = 'Survived'

from sklearn.model_selection import train_test_split

# Preprocessing fills NA values and adds interactions
processed_data = preprocess_titanic(train_data, categorical_cols, numerical_cols)
# display(processed_data)

features = [c for c in train_data.columns.values if c not in ['PassengerId','Survived','Name','Ticket']]

X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data[target], test_size=0.33, random_state=42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Pclass_Sex,Pclass_Cabin,Pclass_Embarked,Sex_Cabin,Sex_Embarked,Cabin_Embarked
PassengerId,,,,,,,,,,,,,,
7,1,male,54.0,0,0,51.8625,E46,S,1_male,1_E46,1_S,male_E46,male_S,E46_S
719,3,male,-1.0,0,0,15.5000,NotSpecified,Q,3_male,3_NotSpecified,3_Q,male_NotSpecified,male_Q,NotSpecified_Q
686,2,male,25.0,1,2,41.5792,NotSpecified,C,2_male,2_NotSpecified,2_C,male_NotSpecified,male_C,NotSpecified_C
74,3,male,26.0,1,0,14.4542,NotSpecified,C,3_male,3_NotSpecified,3_C,male_NotSpecified,male_C,NotSpecified_C
883,3,female,22.0,0,0,10.5167,NotSpecified,S,3_female,3_NotSpecified,3_S,female_NotSpecified,female_S,NotSpecified_S


As stated above, we can use the custom Selector classes for making transforms for specific columns. For example, standardization.
### 2/4/20 notes
After doing some work, I don't think the selector is really necesarry, at least for our case? We can just scale everything once in the pipeline specifying the columns we want to use it on.

Also, in our helper pre-processer, we are filling in NA's. In the future, we can either decide to do that IN the pipeline or not. Maybe better to do in pipeline? But then also how do you do interactions in a pipeline.

#### *Hey*!
we can use scikits [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to create interactions automatically, meaning we can do it in the pipeline! (i think)

In [ ]:
from sklearn.preprocessing import StandardScaler

numerical_tranformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [24]:
# http://scipy-lectures.org/packages/statistics/index.html
# import seaborn as sns

# plot = sns.pairplot(train_data, kind="reg", aspect=1, height=6)
# plot.savefig('output.png')

- Fare needs to have it's NA values filled
- Age NA values need to be handled...